### Challenge 1

- Open up a new IPython notebook  
- Download a few MTA turnstile data files  
- Open up a file, use csv reader to read it, make a python dict where there is a key for each (C/A, UNIT, SCP, STATION). These are the first four columns. The value for this key should be a list of lists. Each list in the list is the rest of the columns in a row. For example, one key-value pair should look like  


{ ('A002','R051','02-00-00','LEXINGTON AVE'): [ ['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'], ['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'], ['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'], ... ] }  

In [1]:
#going to the MTA website and appending multiple data file into the pandas dataframe (for the month of May 2018.

import pandas as pd

links = ['http://web.mta.info/developers/data/nyct/turnstile/turnstile_180505.txt', 
         'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180512.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180519.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180526.txt',
        'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180602.txt']

df = pd.concat([pd.read_csv(l) for l in links], ignore_index=True)

### Challenge 2

- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and
 station of a specific turnstile), have a list again, but let the list
 be comprised of just the point in time and the count of entries.

This basically means keeping only the date, time, and entries fields
in each list. You can convert the date and time into datetime objects
-- That is a python class that represents a point in time. You can
combine the date and time fields into a string and use the
[dateutil](https://labix.org/python-dateutil) module to convert it
into a datetime object. For an example check
[this StackOverflow question](http://stackoverflow.com/questions/23385003/attributeerror-when-using-import-dateutil-and-dateutil-parser-parse-but-no).

Your new dict should look something like

{    ('A002','R051','02-00-00','LEXINGTON AVE'):
[
[datetime.datetime(2013, 3, 2, 3, 0), 3788],
[datetime.datetime(2013, 3, 2, 7, 0), 2585],
[datetime.datetime(2013, 3, 2, 12, 0), 10653],
[datetime.datetime(2013, 3, 2, 17, 0), 11016],
[datetime.datetime(2013, 3, 2, 23, 0), 10666],
[datetime.datetime(2013, 3, 3, 3, 0), 10814],
[datetime.datetime(2013, 3, 3, 7, 0), 10229],
...
],
....
}



In [2]:
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [3]:
df.columns = df.columns.str.strip()
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [4]:
#Concatenating Data and Time before using the parse function from the dateutil package
df['DATE_TIME'] = df['DATE'] + ' ' + df['TIME']
#df.head()
#df.dtypes

In [5]:
#using a for loop to convert date time into datetime/ timeseries.
#eventhough this is slower than the solution that was provided but I thought I'd stick with what I had originally done. 
from datetime import datetime
from dateutil.parser import parse
 
#creates a list comprehension to convert Date and Datetime to 'datetime'
date = [parse(x) for x in df['DATE']] 
parsed_datetime = [parse(x) for x in df['DATE_TIME']]



In [14]:
#df.dtypes
#date[0:4]
#parsed_datetime[0:4]

C/A          object
UNIT         object
SCP          object
STATION      object
LINENAME     object
DIVISION     object
DATE         object
TIME         object
DESC         object
ENTRIES       int64
EXITS         int64
DATE TIME    object
DATE_TIME    object
dtype: object

In [6]:
#adding two new columns in the Dataframe 'df' as datetime64
df['PARSED_DATE'] = date
df['PARSED_DATE_TIME']= parsed_datetime
#df.dtypes
#df.head()

In [7]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,PARSED_DATE,PARSED_DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,00:00:00,REGULAR,6598847,2235829,04/28/2018 00:00:00,2018-04-28,2018-04-28 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,04:00:00,REGULAR,6598864,2235830,04/28/2018 04:00:00,2018-04-28,2018-04-28 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,08:00:00,REGULAR,6598880,2235863,04/28/2018 08:00:00,2018-04-28,2018-04-28 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,12:00:00,REGULAR,6598961,2235955,04/28/2018 12:00:00,2018-04-28,2018-04-28 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2018,16:00:00,REGULAR,6599175,2236015,04/28/2018 16:00:00,2018-04-28,2018-04-28 16:00:00


### Challenge 3

- These counts are for every n hours. (What is n?) We want total daily
  entries.

Now make it that we again have the same keys, but now we have a single
value for a single day, which is the total number of passengers that
entered through this turnstile on this day.

#### Answer. 
**'n' is hours incremental in four hours for each turnstile**. 

In [8]:
dup_df = df[['C/A','UNIT','SCP','STATION','DESC','ENTRIES','EXITS','PARSED_DATE','PARSED_DATE_TIME']]

In [9]:
dup_df.groupby(['C/A','UNIT','SCP','STATION','PARSED_DATE_TIME'])[['ENTRIES']].count().reset_index() \
.sort_values('ENTRIES', ascending=False).head()

,C/A,UNIT,SCP,STATION,PARSED_DATE_TIME,ENTRIES
615668,R101,R001,02-00-05,SOUTH FERRY,2018-05-23 09:00:00,2
615038,R101,R001,02-00-02,SOUTH FERRY,2018-05-23 09:00:00,2
614618,R101,R001,02-00-00,SOUTH FERRY,2018-05-23 09:00:00,2
0,A002,R051,02-00-00,59 ST,2018-04-28 00:00:00,1
656937,R143,R032,02-00-01,TIMES SQ-42 ST,2018-05-14 00:00:00,1


In [10]:
dup_df[(df['C/A'] == 'R101') & (df['UNIT'] == 'R001') & (df['SCP'] == '02-00-05') & (df['STATION'] == 'SOUTH FERRY') \
   & (df['PARSED_DATE_TIME'] == '2018-05-23 09:00:00')]

,C/A,UNIT,SCP,STATION,DESC,ENTRIES,EXITS,PARSED_DATE,PARSED_DATE_TIME
714372,R101,R001,02-00-05,SOUTH FERRY,REGULAR,2582231,1754670,2018-05-23,2018-05-23 09:00:00
714373,R101,R001,02-00-05,SOUTH FERRY,RECOVR AUD,2582231,1754669,2018-05-23,2018-05-23 09:00:00


In [11]:
may_mta_df = dup_df.drop_duplicates(subset=['C/A','UNIT','SCP','STATION','PARSED_DATE_TIME'])

In [12]:
may_mta_df.groupby(['C/A','UNIT','SCP','STATION','PARSED_DATE_TIME'])[['ENTRIES']].count().reset_index() \
.sort_values('ENTRIES', ascending=False).head()

,C/A,UNIT,SCP,STATION,PARSED_DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2018-04-28 00:00:00,1
657012,R143,R032,02-00-01,TIMES SQ-42 ST,2018-05-26 12:00:00,1
656924,R143,R032,02-00-01,TIMES SQ-42 ST,2018-05-11 20:00:00,1
656925,R143,R032,02-00-01,TIMES SQ-42 ST,2018-05-12 00:00:00,1
656926,R143,R032,02-00-01,TIMES SQ-42 ST,2018-05-12 04:00:00,1


In [13]:
#may_mta_df[(df['C/A'] == 'R101') & (df['UNIT'] == 'R001') & (df['SCP'] == '02-00-00') & (df['STATION'] == 'SOUTH FERRY') \
#   & (df['PARSED_DATE_TIME'] == '2018-05-23 09:00:00')]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,C/A,UNIT,SCP,STATION,DESC,ENTRIES,EXITS,PARSED_DATE,PARSED_DATE_TIME
714160,R101,R001,02-00-00,SOUTH FERRY,REGULAR,2967614,4861484,2018-05-23,2018-05-23 09:00:00
